In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader

# BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

# SmallResNet 클래스 정의
class SmallResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=1):
        super(SmallResNet, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=1)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=1)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=1)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.linear(out)
        return out

def SmallResNet18():
    return SmallResNet(BasicBlock, [2, 2, 2, 2])

model = SmallResNet18().to(device)
model = torch.load('model_v5.pt')

In [6]:
import os
from PIL import Image
from torchvision import transforms
import torch

directory = r'C:\Users\lwj01\HowFastTennisBallIs\novak_sinner_over_30'
save_directory = r'C:\Users\lwj01\HowFastTennisBallIs\y_v5'  # 저장할 경로
save_directory_over=r'C:\Users\lwj01\HowFastTennisBallIs\y_v5\over'

def sliding_window(image, step_size, window_size):
    """이미지 위에서 슬라이딩 윈도우를 생성하는 함수"""
    for y in range(0, image.size[1] - window_size[1], step_size):
        for x in range(0, image.size[0] - window_size[0], step_size):
            yield (x, y, image.crop((x, y, x + window_size[0], y + window_size[1])))

def detect_objects_per_image(model, image_path, window_size=(30, 30), step_size=10):
    model.eval()  # 모델을 평가 모드로 설정
    transform = transforms.Compose([transforms.Resize(window_size), transforms.ToTensor()])

    image = Image.open(image_path).convert('RGB')
    image_filename = os.path.basename(image_path).split('.')[0]  # 확장자를 제외한 파일 이름

    for (x, y, window) in sliding_window(image, step_size, window_size):
        window_tensor = transform(window).unsqueeze(0).to(device)
        output = model(window_tensor)
        
        score = torch.sigmoid(output).item()
        window_filename = f"{image_filename}_x{x}_y{y}_score{float(score*100)}.jpg"  # 고유한 파일명 생성

        if score >= 0.5:
            save_path = os.path.join(save_directory_over, window_filename)
        else:
            save_path = os.path.join(save_directory, window_filename)

        window.save(save_path)

# 모든 이미지에 대해 객체 탐지 실행
for filename in os.listdir(directory):
    if filename.endswith('.jpg'):
        image_path = os.path.join(directory, filename)
        score, location, window = detect_objects_per_image(model, image_path)
        print(f"Image: {filename}, Best Score: {score}, Location: {location}")


KeyboardInterrupt: 